<a href="https://colab.research.google.com/github/hala-sdev/game-recommender-system/blob/main/Gaming_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Description:  Develop a recommendation system that suggests video games to users based on their playing history, preferences, and game features.

## **Downloaded Libraries**

In [ ]:
pip install faiss-cpu
nltk.download('punkt')
nltk.download('stopwords')

## **Libraries Imported**

In [ ]:
import pandas as pd
import numpy as np
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MinMaxScaler
from gensim.models import Word2Vec
import faiss
from scipy.sparse import hstack, csr_matrix

## **Data Pre-processing**

In [ ]:
data = pd.read_csv('/content/steam-games.csv')

In [ ]:
data

,app_id,title,release_date,genres,categories,developer,publisher,original_price,discount_percentage,discounted_price,...,win_support,mac_support,linux_support,awards,overall_review,overall_review_%,overall_review_count,recent_review,recent_review_%,recent_review_count
0,730,Counter-Strike 2,21-Aug-12,"Action, Free to Play","Cross-Platform Multiplayer, Steam Trading Card...",Valve,Valve,NaN,NaN,Free,...,True,False,True,1,Very Positive,87.0,8062218.0,Mostly Positive,79.0,57466.0
1,570,Dota 2,09-Jul-13,"Action, Strategy, Free to Play","Steam Trading Cards, Steam Workshop, SteamVR C...",Valve,Valve,NaN,NaN,Free,...,True,True,True,0,Very Positive,81.0,2243112.0,Mostly Positive,72.0,23395.0
2,2215430,Ghost of Tsushima DIRECTOR'S CUT,16-May-24,"Action, Adventure","Single-player, Online Co-op, Steam Achievement...",Sucker Punch Productions,PlayStation PC LLC,NaN,NaN,"₹3,999.00",...,True,False,False,0,Very Positive,89.0,12294.0,NaN,NaN,NaN
3,1245620,ELDEN RING,24-Feb-22,"Action, RPG","Single-player, Online PvP, Online Co-op, Steam...",FromSoftware Inc.,FromSoftware Inc.,NaN,NaN,"₹3,599.00",...,True,False,False,6,Very Positive,93.0,605191.0,Very Positive,94.0,7837.0
4,1085660,Destiny 2,01-Oct-19,"Action, Adventure, Free to Play","Single-player, Online PvP, Online Co-op, Steam...",Bungie,Bungie,NaN,NaN,Free,...,True,False,False,0,Very Positive,80.0,594713.0,Mostly Positive,73.0,4845.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42492,477910,Wanderer of Teandria,26-Sep-17,"Action, Adventure, Indie","Single-player, Steam Achievements, Steam Tradi...",Silentplaygames,Silentplaygames,₹250.00,-50%,₹125.00,...,True,False,False,0,NaN,NaN,NaN,NaN,NaN,NaN
42493,1501390,KING PONG,18-Feb-21,"Action, Sports, Early Access","Single-player, Online PvP, Steam Achievements,...",Iconik,Iconik,₹529.00,-50%,₹264.00,...,True,False,False,1,NaN,NaN,NaN,NaN,NaN,NaN
42494,2683250,Falnarion Tactics III,25-Dec-23,Strategy,"Single-player, Steam Achievements, Steam Cloud...",Team Syukino,Team Syukino,NaN,NaN,₹610.00,...,True,False,False,0,NaN,NaN,NaN,NaN,NaN,NaN
42495,1508840,Great Exploration VR: New Colony beyond Viking...,12-Jan-21,"Action, Casual","Single-player, Tracked Controller Support, VR ...",William at Oxford,William at Oxford,₹199.00,-51%,₹97.00,...,True,False,False,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42497 entries, 0 to 42496
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   app_id                42497 non-null  int64  
 1   title                 42497 non-null  object 
 2   release_date          42440 non-null  object 
 3   genres                42410 non-null  object 
 4   categories            42452 non-null  object 
 5   developer             42307 non-null  object 
 6   publisher             42286 non-null  object 
 7   original_price        4859 non-null   object 
 8   discount_percentage   4859 non-null   object 
 9   discounted_price      42257 non-null  object 
 10  dlc_available         42497 non-null  int64  
 11  age_rating            42497 non-null  int64  
 12  content_descriptor    2375 non-null   object 
 13  about_description     42359 non-null  object 
 14  win_support           42497 non-null  bool   
 15  mac_support        

In [ ]:
data_up = data.drop_duplicates()
data_up = data_up.drop(columns=['discount_percentage', 'discounted_price', 'original_price', 'awards','recent_review_%', 'recent_review_count','recent_review','dlc_available','content_descriptor','release_date'])
data_up

,app_id,title,genres,categories,developer,publisher,age_rating,about_description,win_support,mac_support,linux_support,overall_review,overall_review_%,overall_review_count
0,730,Counter-Strike 2,"Action, Free to Play","Cross-Platform Multiplayer, Steam Trading Card...",Valve,Valve,0,"For over two decades, Counter-Strike has offer...",True,False,True,Very Positive,87.0,8062218.0
1,570,Dota 2,"Action, Strategy, Free to Play","Steam Trading Cards, Steam Workshop, SteamVR C...",Valve,Valve,0,"Every day, millions of players worldwide enter...",True,True,True,Very Positive,81.0,2243112.0
2,2215430,Ghost of Tsushima DIRECTOR'S CUT,"Action, Adventure","Single-player, Online Co-op, Steam Achievement...",Sucker Punch Productions,PlayStation PC LLC,1,A storm is coming. Venture into the complete G...,True,False,False,Very Positive,89.0,12294.0
3,1245620,ELDEN RING,"Action, RPG","Single-player, Online PvP, Online Co-op, Steam...",FromSoftware Inc.,FromSoftware Inc.,1,"THE NEW FANTASY ACTION RPG. Rise, Tarnished, a...",True,False,False,Very Positive,93.0,605191.0
4,1085660,Destiny 2,"Action, Adventure, Free to Play","Single-player, Online PvP, Online Co-op, Steam...",Bungie,Bungie,1,Destiny 2 is an action MMO with a single evolv...,True,False,False,Very Positive,80.0,594713.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42492,477910,Wanderer of Teandria,"Action, Adventure, Indie","Single-player, Steam Achievements, Steam Tradi...",Silentplaygames,Silentplaygames,0,Wanderer of Teandria is challenging 2D platfor...,True,False,False,NaN,NaN,NaN
42493,1501390,KING PONG,"Action, Sports, Early Access","Single-player, Online PvP, Steam Achievements,...",Iconik,Iconik,0,If you've always dreamed of being a table tenn...,True,False,False,NaN,NaN,NaN
42494,2683250,Falnarion Tactics III,Strategy,"Single-player, Steam Achievements, Steam Cloud...",Team Syukino,Team Syukino,0,"A missing queen, a world engulfed in war, a my...",True,False,False,NaN,NaN,NaN
42495,1508840,Great Exploration VR: New Colony beyond Viking...,"Action, Casual","Single-player, Tracked Controller Support, VR ...",William at Oxford,William at Oxford,0,"In the year of our lord 1453, the hundred year...",True,False,False,NaN,NaN,NaN


In [ ]:
data['about_description'] = data['about_description'].fillna('')


stop_w = set(stopwords.words('english'))
def remove_sc(text):
    words = word_tokenize(text)
    filtered_words = [w for w in words if w.isalpha()]
    return ' '.join(filtered_words)

def remove_stopwords_sc(text):
    words = word_tokenize(text)
    filtered_words = [w for w in words if w.lower() not in stop_w]
    filtered_words = [w for w in filtered_words if w.isalpha()]
    return ' '.join(filtered_words)

data['about_description'] = data['about_description'].apply(remove_stopwords_sc)
print(data['about_description'])


0        two decades offered elite competitive experien...
1        Every day millions players worldwide enter bat...
2        storm coming Venture complete Ghost Tsushima D...
3        NEW FANTASY ACTION RPG Rise Tarnished guided g...
4        Destiny action MMO single evolving world frien...
                               ...                        
42492    Wanderer Teandria challenging platform game in...
42493    always dreamed table tennis king successfully ...
42494    missing queen world engulfed war mysterious ne...
42495    year lord hundred year war England France fina...
42496    SANCTION procedurally generated world prisoner...
Name: about_description, Length: 42497, dtype: object


In [ ]:
def tokenizer(text):
  return word_tokenize(text)

data['about_description'] = data['about_description'].apply(tokenizer)
print(data['about_description'])

0        [two, decades, offered, elite, competitive, ex...
1        [Every, day, millions, players, worldwide, ent...
2        [storm, coming, Venture, complete, Ghost, Tsus...
3        [NEW, FANTASY, ACTION, RPG, Rise, Tarnished, g...
4        [Destiny, action, MMO, single, evolving, world...
                               ...                        
42492    [Wanderer, Teandria, challenging, platform, ga...
42493    [always, dreamed, table, tennis, king, success...
42494    [missing, queen, world, engulfed, war, mysteri...
42495    [year, lord, hundred, year, war, England, Fran...
42496    [SANCTION, procedurally, generated, world, pri...
Name: about_description, Length: 42497, dtype: object


In [ ]:
data['about_description'] = data['about_description'].astype(str)
data['genres'] = data['genres'].astype(str)
data['categories'] = data['categories'].astype(str)
data['categories'] = data['categories'].apply(remove_sc)
data['categories'] = data['categories'].fillna('')
data['categories'] = data['categories'].apply(tokenizer)


In [ ]:
data['genres'] = data['genres'].apply(remove_sc)
data['genres'] = data['genres'].fillna('')
data['genres'] = data['genres'].apply(tokenizer)
print(data['categories'])
print(data['genres'])

0        [Multiplayer, Steam, Trading, Cards, Steam, Wo...
1        [Steam, Trading, Cards, Steam, Workshop, Steam...
2           [Online, Steam, Achievements, Family, Sharing]
3        [Online, PvP, Online, Steam, Achievements, Ste...
4        [Online, PvP, Online, Steam, Achievements, Pur...
                               ...                        
42492    [Steam, Achievements, Steam, Trading, Cards, S...
42493    [Online, PvP, Steam, Achievements, Tracked, Co...
42494    [Steam, Achievements, Steam, Cloud, Family, Sh...
42495    [Tracked, Controller, Support, VR, Only, Famil...
42496    [Steam, Achievements, Family, Sharing, Profile...
Name: categories, Length: 42497, dtype: object
0                                 [Action, Free, to, Play]
1                       [Action, Strategy, Free, to, Play]
2                                      [Action, Adventure]
3                                            [Action, RPG]
4                      [Action, Adventure, Free, to, Play]
         

In [ ]:
data['overall_review_%'] = data['overall_review_%'].fillna(0)
data['overall_review_count'] = data['overall_review_count'].fillna(0)
scale = MinMaxScaler()
data[['overall_review_%', 'overall_review_count','win_support','mac_support','linux_support']] = scale.fit_transform(data[['overall_review_%', 'overall_review_count','win_support','mac_support','linux_support']])
print(data['overall_review_%'])
print(data['overall_review_count'])

0        0.87
1        0.81
2        0.89
3        0.93
4        0.80
         ... 
42492    0.00
42493    0.00
42494    0.00
42495    0.00
42496    0.00
Name: overall_review_%, Length: 42497, dtype: float64
0        1.000000
1        0.278225
2        0.001525
3        0.075065
4        0.073765
           ...   
42492    0.000000
42493    0.000000
42494    0.000000
42495    0.000000
42496    0.000000
Name: overall_review_count, Length: 42497, dtype: float64


In [ ]:
data['about_description'] = data['about_description'].astype(str)
data['genres'] = data['genres'].astype(str)
data['categories'] = data['categories'].astype(str)
data['about_description'] = data['about_description'].str.lower()
data['genres'] = data['genres'].str.lower()
data['categories'] = data['categories'].str.lower()
data['about_description']

0        ['two', 'decades', 'offered', 'elite', 'compet...
1        ['every', 'day', 'millions', 'players', 'world...
2        ['storm', 'coming', 'venture', 'complete', 'gh...
3        ['new', 'fantasy', 'action', 'rpg', 'rise', 't...
4        ['destiny', 'action', 'mmo', 'single', 'evolvi...
                               ...                        
42492    ['wanderer', 'teandria', 'challenging', 'platf...
42493    ['always', 'dreamed', 'table', 'tennis', 'king...
42494    ['missing', 'queen', 'world', 'engulfed', 'war...
42495    ['year', 'lord', 'hundred', 'year', 'war', 'en...
42496    ['sanction', 'procedurally', 'generated', 'wor...
Name: about_description, Length: 42497, dtype: object

##  **Word2Vec Embedding for Descriptions**

In [ ]:

def average_wv(model, words):
    word_vectors = [model.wv[w] for w in words if w in model.wv]
    if not word_vectors:
        np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)


#description requires semantic
dsc_model = Word2Vec(sentences=data['about_description'], vector_size=100, window=5, min_count=1, workers=4)
data['about_description'] = data['about_description'].apply(lambda x: average_wv(dsc_model, x))
print(data['about_description'])

#other require tidif

0        [0.13674483, -0.24585596, -0.25302595, -0.6387...
1        [0.15633023, -0.24375401, -0.25604612, -0.6529...
2        [0.12027196, -0.25234297, -0.25700387, -0.6360...
3        [0.13988367, -0.26267347, -0.22586356, -0.6536...
4        [0.11325613, -0.2186791, -0.22044605, -0.62403...
                               ...                        
42492    [0.12860887, -0.24023658, -0.24628663, -0.6408...
42493    [0.13952902, -0.25254005, -0.2178578, -0.63479...
42494    [0.12643236, -0.25610873, -0.22798765, -0.6597...
42495    [0.16979288, -0.25828516, -0.22166623, -0.6674...
42496    [0.13525033, -0.22885247, -0.25123018, -0.6365...
Name: about_description, Length: 42497, dtype: object


## **TF-IDF Vectors for Genres & Categories**


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tfidf = TfidfVectorizer()

tfidf_genre = tfidf.fit_transform(data['genres'])
tfidf_category = tfidf.fit_transform(data['categories'])



In [ ]:
data['genres']

0                         ['action', 'free', 'to', 'play']
1             ['action', 'strategy', 'free', 'to', 'play']
2                                  ['action', 'adventure']
3                                        ['action', 'rpg']
4            ['action', 'adventure', 'free', 'to', 'play']
                               ...                        
42492                     ['action', 'adventure', 'indie']
42493              ['action', 'sports', 'early', 'access']
42494                                         ['strategy']
42495                                 ['action', 'casual']
42496    ['action', 'adventure', 'casual', 'indie', 'si...
Name: genres, Length: 42497, dtype: object

## **Recommendation Function**

In [ ]:

description_vectors = np.vstack(data['about_description'].values)
description_vectors_sparse = csr_matrix(description_vectors)

combined_results = hstack([tfidf_genre, tfidf_category, description_vectors_sparse])
combined_results_arr = combined_results.toarray()

index = faiss.IndexFlatL2(combined_results_arr.shape[1])
index.add(combined_results_arr)

# Function to find the index of a game by title
def game_indx(title, data):
    title = title.lower()
    data['title'] = data['title'].str.lower()
    result = data[data['title'] == title]
    if result.empty:
        raise ValueError(f"The Game '{title}' is not found in dataset.")
    return result.index[0]

specific_game_title = 'Danganronpa: Trigger Happy Havoc'
specific_game_index = game_indx(specific_game_title, data)

D, I = index.search(np.array([combined_results_arr[specific_game_index]]), 10)
recommended_titles = data.iloc[I[0]]['title'].values

print(f"Recommendations for the game {specific_game_title}:")
for title in recommended_titles:
    print(title)

Recommendations for the game Danganronpa: Trigger Happy Havoc:
danganronpa: trigger happy havoc
little nightmares ii
chaos;child
phoenix wright: ace attorney trilogy
umineko when they cry - answer arcs
master magistrate
danganronpa 2: goodbye despair
umineko when they cry - question arcs
steins;gate
ai: the somnium files
